# The bellow is a look at how evals work in the contest

The following is from the contest itself:

"For each row in the submission and corresponding ground truth, sentence-t5-base is used to calculate corresponding embedding vectors. The score for each predicted / expected pair is calculated using the Sharpened Cosine Similarity, using an exponent of 3. The SCS is used to attenuate the generous score given by embedding vectors for incorrect answers. Do not leave any rewrite_prompt blank as null answers will throw an error." ~ https://www.kaggle.com/competitions/llm-prompt-recovery/overview

In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
st_model = SentenceTransformer('sentence-transformers/sentence-t5-base')

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/rust_model.ot:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [16]:
sentences = [
    "Transform this sentence into a more humorous and sarcastic version, using irony and wit.",
    "Rewrite this text to infuse it with humor.",
    "Rewrite this text to infuse it with humor",
    "Transform this to be more humorous"
]
sentence_embeddings = st_model.encode(sentences)

scores = cosine_similarity(sentence_embeddings, sentence_embeddings) 

sharpened_scores = scores ** 3
sharpened_scores

# Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(sharpened_scores.shape[0]):
    for j in range(sharpened_scores.shape[1]):
        pairs.append({"index": [i, j], "score": sharpened_scores[i][j]})

# Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x["score"], reverse=True)

for pair in pairs[0:10]:
    i, j = pair["index"]
    print("{} \t\t {} \t\t Score: {:.4f}".format(
        sentences[i], sentences[j], pair["score"]
    ))

Rewrite this text to infuse it with humor. 		 Transform this to be more humorous 		 Score: 0.7163
Rewrite this text to infuse it with humor. 		 Rewrite this text to infuse it with humor 		 Score: 0.7111
Rewrite this text to infuse it with humor. 		 Rewrite this text to infuse it with humor. 		 Score: 0.7044


In [28]:
gt_sentences = [
    "Transform this sentence into a more humorous and sarcastic version, using irony and wit.",
]
sentences = [
    "Transform this sentence into a more humorous",
    "Rewrite this text to infuse it with humor.",
    "Rewrite this text to infuse it with humor",
    "Transform this to be more humorous",
    "Rewrite this to be more humerous",
    "Rewrite this text to infuse it with humor and a light-hearted tone, while still conveying the scientific facts and the relative safety due to the distant future impact date."
    "Refine the following passage by emulating the writing style of [insert desired style here], with a focus on enhancing its clarity, elegance, and overall impact. Preserve the essence and original meaning of the text, while meticulously adjusting its tone, vocabulary, and stylistic elements to resonate with the chosen style.Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style."
]

sentence_embeddings = st_model.encode(sentences)
gt_embeddings = st_model.encode(gt_sentences)

scores = cosine_similarity(gt_embeddings, sentence_embeddings) 

sharpened_scores = scores ** 3

# Output the pairs with their score
for i in range(len(sentences)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(
        gt_sentences[0], sentences[i], sharpened_scores[0][i]
    ))

Transform this sentence into a more humorous and sarcastic version, using irony and wit. 		 Transform this sentence into a more humorous 		 Score: 0.7927
Transform this sentence into a more humorous and sarcastic version, using irony and wit. 		 Rewrite this text to infuse it with humor. 		 Score: 0.7044
Transform this sentence into a more humorous and sarcastic version, using irony and wit. 		 Rewrite this text to infuse it with humor 		 Score: 0.7111
Transform this sentence into a more humorous and sarcastic version, using irony and wit. 		 Transform this to be more humorous 		 Score: 0.7163
Transform this sentence into a more humorous and sarcastic version, using irony and wit. 		 Rewrite this to be more humerous 		 Score: 0.7245
Transform this sentence into a more humorous and sarcastic version, using irony and wit. 		 Rewrite this text to infuse it with humor and a light-hearted tone, while still conveying the scientific facts and the relative safety due to the distant future impa

In [7]:
predicted_sentence = "This framework generates embeddings for each input sentence"
expected_sentence = "Sentences are passed as a list of string."

# Calculate embedding vectors for each sentence
predicted_embeddings = st_model.encode(predicted_sentence)
expected_embeddings = st_model.encode(expected_sentence)


In [9]:
predicted_embeddings

array([-1.66203845e-02,  3.18243094e-02,  1.08900415e-02,  6.07051626e-02,
       -4.78123827e-03,  1.11913783e-02, -6.85976222e-02,  4.61351424e-02,
       -4.64002341e-02, -1.12267621e-02,  2.83614602e-02, -4.56234030e-02,
        7.72971362e-02,  3.75814810e-02,  6.19152375e-02,  4.86828275e-02,
        1.76920649e-02, -5.03368815e-03,  4.12636138e-02,  2.81515792e-02,
        6.77129477e-02, -5.78775108e-02, -1.62556749e-02,  1.80227738e-02,
       -3.35297063e-02,  1.58178154e-02,  3.43615450e-02,  9.60681122e-03,
       -4.21833480e-03,  1.61159895e-02,  9.31848958e-02, -4.50574942e-02,
       -2.59352662e-02,  6.35581762e-02, -2.26409975e-02,  3.84901017e-02,
       -1.16953626e-02,  4.39094426e-03, -4.13329415e-02,  3.33792083e-02,
       -1.83012262e-02, -4.07681465e-02, -5.73155209e-02,  6.27704486e-02,
       -3.43312472e-02, -3.07545476e-02,  2.32101697e-02,  9.06041935e-02,
       -8.93085729e-03, -7.34184682e-02, -1.31707033e-02, -2.64743157e-02,
       -4.25296910e-02, -

In [8]:

# Calculate cosine similarity scores
scores = cosine_similarity(predicted_embeddings, expected_embeddings)

# Apply Sharpened Cosine Similarity with exponent of 3
sharpened_scores = scores ** 3
sharpened_scores

ValueError: Expected 2D array, got 1D array instead:
array=[-1.66203845e-02  3.18243094e-02  1.08900415e-02  6.07051626e-02
 -4.78123827e-03  1.11913783e-02 -6.85976222e-02  4.61351424e-02
 -4.64002341e-02 -1.12267621e-02  2.83614602e-02 -4.56234030e-02
  7.72971362e-02  3.75814810e-02  6.19152375e-02  4.86828275e-02
  1.76920649e-02 -5.03368815e-03  4.12636138e-02  2.81515792e-02
  6.77129477e-02 -5.78775108e-02 -1.62556749e-02  1.80227738e-02
 -3.35297063e-02  1.58178154e-02  3.43615450e-02  9.60681122e-03
 -4.21833480e-03  1.61159895e-02  9.31848958e-02 -4.50574942e-02
 -2.59352662e-02  6.35581762e-02 -2.26409975e-02  3.84901017e-02
 -1.16953626e-02  4.39094426e-03 -4.13329415e-02  3.33792083e-02
 -1.83012262e-02 -4.07681465e-02 -5.73155209e-02  6.27704486e-02
 -3.43312472e-02 -3.07545476e-02  2.32101697e-02  9.06041935e-02
 -8.93085729e-03 -7.34184682e-02 -1.31707033e-02 -2.64743157e-02
 -4.25296910e-02 -2.83144843e-02 -6.20349869e-02 -3.36020924e-02
 -7.34354556e-03  2.17808504e-0